<a href="https://colab.research.google.com/github/mowne67/Portfolio-Mowne/blob/main/Television_Image_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Television Image Detector using Tensorflow Keras
An image processing neural network is designed that takes any image from the internet and detects the presence of a television in the image. The ***model is trained using a dataset that is created from scratch*** from a webscraper notebook which outputs a csv file of image URLs from DuckDuckGo search engine images.


In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

Two datasets were created using the Imagescraper. The `random.csv` file contains URLs of random images (not television) that are labelled as `0`. The `tvimage.csv` file contains URLs of images of television that are labelled as `1`.







In [5]:
random=pd.read_csv("random.csv")
tvimage=pd.read_csv("tvimage.csv")

The two datasets are combined to create a new DataFrame `tvtrain` which is going to be used for training.

In [8]:
tvtrain=pd.DataFrame()
tvtrain["URL"]=list(random["URL"])+list(tvimage["URL"])
tvtrain["Label"]=list(random["Label"])+list(tvimage["Label"])

In [10]:
tvtrain

,URL,Label
0,https://tse3.mm.bing.net/th?id=OIP.E0IO_W_4xEl...,0
1,https://tse4.mm.bing.net/th?id=OIP.L-Uzd8nIU6G...,0
2,https://tse3.mm.bing.net/th?id=OIP.0N_-pJLPt0l...,0
3,https://tse3.explicit.bing.net/th?id=OIP.Gqhlz...,0
4,https://tse1.mm.bing.net/th?id=OIP.HSPjjE1j3wD...,0
...,...,...
845,https://tse2.mm.bing.net/th?id=OIP.MuQkn-8N5vJ...,1
846,https://tse1.mm.bing.net/th?id=OIP.c_JH2O92x9G...,1
847,https://tse4.mm.bing.net/th?id=OIP.9ay-VVnljLF...,1
848,https://tse4.mm.bing.net/th?id=OIP._Jc2jHL6eBB...,1


Testing out the DataFrame:

In [11]:
tvimage["Label"][440]

1

In [12]:
tvimage['URL'][45]

'https://tse3.mm.bing.net/th?id=OIP.4xGA1b_IhOYC_GUzB87vkwHaHa&pid=Api'

##Image Pre-Processing

As these URLs contain the URLs of the jpeg files, `requests` library is used to open the jpeg image from the URL. Then, the jpeg file is converted into a numpy array with the size of `(800,400,3)` where the 3 is for the RGB dimension. To complete these tasks for all the URLs in the tvtrain dataframe, the function `preprocess` is created.




In [13]:
from PIL import Image
import requests
from keras.preprocessing import image

In [15]:
def preprocess(url):
  img= Image.open(requests.get(url, stream=True).raw)
  
  img_array = image.img_to_array(img)

  imag = tf.image.resize(img_array, (800,400))
  return imag


A new list called tvtrain_images is created where all the preprocessed images are stored as arrays. The list is converted to a numpy array.

In [16]:
tvtrain_images=[]
for url in tvtrain["URL"]:
  tvtrain_images.append(preprocess(url))
len(tvtrain_images)

850

In [17]:
tvtrain_images=np.array(tvtrain_images)
tvtrain_images.shape

(850, 800, 400, 3)

The labels are stored in a numpy array.

In [18]:
tvtrain_labels=np.array(tvtrain["Label"])
tvtrain_labels=tvtrain_labels.reshape(len(tvtrain_images),1)

In [19]:
tvtrain_labels.shape

(850, 1)

##Neural Network Architecture
A CNN base is used followed by some dense layers. The numpy arrays we created will come in handy here as it is easier to use an input. Using Keras, the model is constructed as given below. There are three convolutional layers with 16,32,32 filters each respectively along with two max pooling layers that pools with (5,5) size.

In [20]:
model = models.Sequential()
model.add(layers.Conv2D(16, (8,8), activation='relu', input_shape=(800,400, 3)))
model.add(layers.MaxPooling2D((5,5)))
model.add(layers.Conv2D(32, (8,8), activation='relu'))
model.add(layers.MaxPooling2D((5,5)))
model.add(layers.Conv2D(32, (8,8), activation='relu'))

The dense part consists of a flattening layer which flattens the output of the final layer of CNN base. It is followed by another dense layer with 32 weights and then the output layer is created finally with a single unit that gives us the binary output of the image being a TV or not.

In [21]:
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1))

###Summary of the Model:

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 793, 393, 16)      3088      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 158, 78, 16)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 151, 71, 32)       32800     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 23, 7, 32)         65568     
_________________________________________________________________
flatten (Flatten)            (None, 5152)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                1

##Training and Testing
The tensors tvtrain_images,tvtrain_labels is split into test and train data (0.15 split). Then the model created is trained with the usage of hyperparameters as stated in the code.

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
xtrain,xtest,ytrain,ytest=train_test_split(tvtrain_images,tvtrain_labels,test_size=0.15,random_state=0)

In [30]:
base_learning_rate = 0.0001
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(xtrain, ytrain, epochs=12, validation_data=(xtest,ytest))

Epoch 1/12
23/23 [==============================] - 5s 195ms/step - loss: 0.3626 - accuracy: 0.8518 - val_loss: 0.8157 - val_accuracy: 0.7812
Epoch 2/12
23/23 [==============================] - 4s 191ms/step - loss: 0.2150 - accuracy: 0.9030 - val_loss: 0.6225 - val_accuracy: 0.8047
Epoch 3/12
23/23 [==============================] - 4s 183ms/step - loss: 0.1109 - accuracy: 0.9598 - val_loss: 0.6653 - val_accuracy: 0.8203
Epoch 4/12
23/23 [==============================] - 4s 180ms/step - loss: 0.0684 - accuracy: 0.9737 - val_loss: 0.9225 - val_accuracy: 0.8281
Epoch 5/12
23/23 [==============================] - 4s 177ms/step - loss: 0.1121 - accuracy: 0.9640 - val_loss: 0.7115 - val_accuracy: 0.8359
Epoch 6/12
23/23 [==============================] - 4s 192ms/step - loss: 0.0899 - accuracy: 0.9709 - val_loss: 1.1453 - val_accuracy: 0.7812
Epoch 7/12
23/23 [==============================] - 4s 178ms/step - loss: 0.1745 - accuracy: 0.9363 - val_loss: 1.1010 - val_accuracy: 0.7266
Epoch 

Our Model has an accuracy of 82.81% which is good for an Image Processing model that only has 850 training images with few layers in the neural network.

In [27]:
model.save("finding tv images.h5") 
new_model = tf.keras.models.load_model('finding tv images.h5')

Testing the model directly with any Image from the Internet:

In [29]:
tryimage=preprocess("") #Input Image URL here
tryimg=np.array(tryimage).reshape(1,800,400,3)
model.predict(tryimg)
# A positive value of higher magnitude usually means that it is a TV.

array([[-15.227043]], dtype=float32)

Use this link to open the [DuckDuckGo ImageScraper](https://colab.research.google.com/drive/16CaL8WJDat8Zjc0hJoK9-9cR41lB572U?usp=sharing)